In [1]:
from collections import defaultdict
from random import randint
from copy import copy

class Worker:
    def __init__(self, id):
        self.worker_id = id

available_games = 10
workers = []
for id in range(3):
    workers.append(Worker(id))
    
worker_record = defaultdict(lambda: [])
worker_blocks = defaultdict(lambda: [])

worker_bans = []

game_id = None

In [2]:
def set_game_id(id):
    """
    Sets the global variable game_id to a given value
    :param id: a new value for global variable game_id
    :return: Nothing.
    """
    global game_id
    # print("Setting game ID to {}".format(game_id))
    game_id = id

def select_random_game(exceptions = None):
    """
    Returns a random game ID
    :return: a random game ID
    """
    global available_games
    while True:
        game_id = randint(0, available_games)
#         print("Game ID is {}. Exceptions are {}".format(game_id, exceptions))
        if exceptions != None:
            if game_id not in exceptions:
                break
        else: break
#     print("Selected random game {}".format(game_id))
    return game_id

def update_records(players, game_id):
    set_game_id(game_id)
    
    # Add game ID to the worker record and to the blocked list if it is the second occurance.
    # If a player has played 10 games, he or she gets banned from the game.
    worker_record[players[0].worker_id].append(game_id)
    if worker_record[players[0].worker_id].count(game_id) == 2:
         worker_blocks[players[0].worker_id].append(game_id)
    if len(worker_record[players[0].worker_id]) == 10:
        worker_bans.append(players[0].worker_id)

    worker_record[players[1].worker_id].append(game_id)
    if worker_record[players[1].worker_id].count(game_id) == 2:
         worker_blocks[players[1].worker_id].append(game_id)
    if len(worker_record[players[1].worker_id]) == 10:
        worker_bans.append(players[1].worker_id)

def check_workers_eligibility(workers):
    """
    Checks the list of available workers, pairs the first two players that pass
    the game criteria and sets the corresponding game id
    :param workers: a list of all available workers
    :return: a list of two workers that are paired for the next game
    """

    # TODO: Add a total game maximum
    
    VERBOSE = False

    global worker_record
    global worker_blocks
    global worker_bans

    players = []
    game_id = None

    if len(workers) > 1:
        if VERBOSE: print("{} workers available:".format(len(workers)))
        
        for idx, worker in enumerate(workers[:-1]):
            worker_id = worker.worker_id
            if VERBOSE: print("Worker: {}".format(worker_id))
            players.append(worker)
            
            # If worker never played before, select the last game from the list
            # of his or her partner (next worker in line)
            if worker_id not in worker_record.keys():
                if VERBOSE: print("Worker has no recorded games")

                partner = workers[idx+1]
                partner_id = partner.worker_id
                players.append(partner)
                if VERBOSE: print("Partner: {}".format(partner_id))

                # If the partner also has no games, start a random game
                if partner_id not in worker_record.keys():
                    game_id = select_random_game()
                    if VERBOSE: print("Partner has no recorded games. Setting game ID randomly to {}".format(game_id))
                    update_records(players, game_id)
                    return players
                # Else select the last free game from partner's list
                else:
                    for played_id in worker_record[partner_id][::-1]:
                        if played_id not in worker_blocks[partner_id]:
                            game_id = played_id
                            players.append(partner)
                            if VERBOSE: print("Partner has recorded games. Setting game ID to {}".format(game_id))
                            update_records(players, game_id)
                            return players

                    # If partner does not have free games, start a random one
                    # with exception of his or her blocked list
                    if not game_id:
                        game_id = select_random_game(exceptions=worker_blocks[partner_id])

                    if VERBOSE: print("Partner has recorded games. Setting game ID to {}".format(game_id))
                    update_records(players, game_id)
                    return players

            # If worker played before, make sure that the partner has never played the same game
            else:
                # Check if the worker has any game that was not yet blocked
                for played_id in worker_record[worker_id][::-1]:
                    if played_id not in worker_blocks[worker_id]:
                        game_id = played_id
                        break

                # If no free game is found, check the games of the partner
                if not game_id:
                    if VERBOSE: print("All worker games are blocked")
                    for partner in workers[idx+1:]:
                        partner_id = partner.worker_id
                        # If partner never played before, select a random game
                        # with exception of the blocked list of worker
                        if partner_id not in worker_record.keys():
                            game_id = select_random_game(exceptions=worker_blocks[worker_id])
                            players.append(partner)
                            if VERBOSE: print("Partner has no recorded games. Setting game ID randomly to {}".format(game_id))
                            update_records(players, game_id)
                            return players
                            
                        # Else check the player's previous games
                        else:
                            for played_id in worker_record[partner_id][::-1]:                                
                                if played_id not in worker_blocks[worker_id] \
                                and played_id not in worker_blocks[partner_id]:
                                    game_id = played_id
                                    break

                            # If partner also has no free games, select a random one with exception of
                            # both players' blocked lists
                            if not game_id:
                                blocked = copy(worker_blocks[worker_id])
                                blocked.extend(worker_blocks[partner_id])
                                if VERBOSE: print(blocked)
                                game_id = select_random_game(exceptions=blocked)
                                
                            players.append(partner)
                            if VERBOSE: print("All recorded games are blocked for pair {}-{}. Setting game ID to {}".format(worker_id, partner_id, game_id))
                            update_records(players, game_id)
                            return players
                                
                # Else match a partner to the found game
                else:
                    if VERBOSE: print("Worker has recorded games. Setting game ID to {}".format(game_id))
                    for partner in workers[idx:]:                          
                        partner_id = partner.worker_id
                        if VERBOSE: print("Checking potential partner {}".format(partner_id))
                        # If partner never played the worker's last game, pair them. Otherwise continue looping
                        if game_id not in worker_record[partner_id]:
                            players.append(partner)
                            if VERBOSE: print("Partner has not played this game before.")
                            update_records(players, game_id)
                            return players
        
        if VERBOSE: print("Houston, we have a problem. All is lost.")
    return []
            
            

    

    

In [7]:
print('Records:')
for key, value in worker_record.items():
    print(key)
    print(value)

print('Blocks:')
for key, value in worker_blocks.items():
    print(key)
    print(value)
    
print('Bans:')
for worker in worker_bans:
    print(worker)

Records:
0
[10, 10]
1
[10]
2
[10]
Blocks:
0
[10]
Bans:


In [8]:
players = check_workers_eligibility(workers)

In [196]:
worker_record = defaultdict(lambda: [])
worker_blocks = defaultdict(lambda: [])